In [244]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [245]:
#######################################################
#                 constants for the evaluators
#######################################################

MIN_MEASUREMENT_NUMBER = 5

In [3]:
#######################################################
#                 in-phase evaluator - mprtp2
#######################################################

import pandas as pd
import numpy as np

class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

packets_colnames = ["Tracked NTP",
                    "Sequence Number",
                    "Timestamp",
                    "SSRC",
                    "Payload Type",
                    "Payload Size",
                    "Subflow ID",
                    "Subflow Sequence Number",
                    "Header Size",
                    "Protect Begin",
                    "Protect End",
                    "Marker",
                   ]

class Subflow:
    def __init__(self, channel, path, subflow_id):
        self._path = path
        self._subflow_id = subflow_id
        self._df = pd.DataFrame()
        sr = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'subflow_'+ str(subflow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }
        
    
def collect_subflows(channel, path, subflows_num):
    subflows = [Subflow(channel, path, subflow_id) for subflow_id in range(1, subflows_num + 1)]
    return subflows

    
import os
from os import listdir
from os.path import isfile, join

configs = [
{
    'base_path': "mprtp2",
    'debug': False,
#     'debug': True,
    'channel': "1"
},
{
    'base_path': "mprtp5",
    'debug': False,
#     'debug': True,
    'channel': "1"
}, 
]

for config in configs:
    print(config)
    for path,dirs,files in os.walk(base_path):
        if len(dirs) < 1:
            continue
        final_table = pd.DataFrame()
    #     for subflows_num in [5]:
        for subflows_num in [1, 2, 3, 5]:
    #         print("number of subflows", subflows_num)
            suffix = "_" + str(subflows_num)
            metrics = {
                'SR': [],
                'ABU': [],
                'LR': [],
                'LRN': [],            
                'QMD': [],
                'GP': [],
            }
            subjects = []
            for directory in dirs:
                if not directory.endswith(suffix):
                    continue
                try:
                    subflows = collect_subflows(config['channel'], config['base_path'] + '/' + directory, subflows_num)
                except:
                    print("There is an exception occured at:", directory)

                abs_ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows)
                abs_LR = sum(subflow.get_summary()['LR'] for subflow in subflows)
                abs_GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
                abs_QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows)
                abs_LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)
                subjects.append((directory, abs_ABU, abs_LR, abs_GP, abs_QMD, abs_LRN))

                SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
                ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) / float(subflows_num)
                LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
                LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
                QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
                GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
                metrics['SR'].append(SR)
                metrics['ABU'].append(ABU)            
                metrics['LR'].append(LR)
                metrics['LRN'].append(LRN)            
                metrics['QMD'].append(QMD)
                metrics['GP'].append(GP)

            # Selection method
            # setup sets for the max 10 based on criteria (LR, ABU, and verifications) and filter out the 
            # intersection of the weakers

            # Display parameters
            if config['debug']:
                index = 1
                if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                    print("for measuring ', subflows_num, 'subflows, \
                          it does not have the necessary minimum number of measurements")
    #             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
                sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
                for item in sorted_list:
                    print("Rank:", index)
                    print("Directory, ABU, LR, GP, QMD, LRN: ", item)
                    pdf = PDF(base_path + '/' + item[0] + '/' + 'mprtp2_aggr_FRACTaL_50ms_0ms.pdf')
                    display(pdf)
                    index += 1
                print('-' * 20)

            subflow_col = (str(subflows_num) + ' Subflows') if 1 < subflows_num else '1 Subflow'
            final_table[subflow_col] = list(range(7))
            SRs = np.asarray(metrics['SR'])
            ABUs = np.asarray(metrics['ABU'])
            LRs = np.asarray(metrics['LR'])
            LRNs = np.asarray(metrics['LRN'])
            QMDs = np.asarray(metrics['QMD'])
            GPs = np.asarray(metrics['GP'])

            final_table[subflow_col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
            final_table[subflow_col][1] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
            final_table[subflow_col][2] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
            final_table[subflow_col][3] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
            final_table[subflow_col][4] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
            final_table[subflow_col][5] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
            final_table[subflow_col][6] = str(len(metrics['SR']))

        final_table = final_table.rename({
            0: 'GP [kbps]',
            1: 'ABU [%]', 
            2: 'LR [%]', 
            3: 'LRN',
            4: 'QMD [ms]',
            5: 'SR [kbps]',
            6: 'Samples'}, axis='index')
    #     display(final_table)
        display(final_table.loc[['GP [kbps]', 'LR [%]', 'QMD [ms]'], :])

    
# We need the number of lost packets instead!!!!!

{'debug': False, 'channel': '1', 'base_path': 'mprtp2'}


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,1 Subflow,2 Subflows,3 Subflows,5 Subflows
GP [kbps],481 ± 42,724 ± 88,1149 ± 81,1791 ± 255
LR [%],1% ± 0%,2% ± 0%,2% ± 0%,2% ± 0%
QMD [ms],135.25 ± 2.72,110.67 ± 1.5,109.92 ± 1.95,109.23 ± 2.64


{'debug': False, 'channel': '1', 'base_path': 'mprtp5'}
('There is an exception occured at:', 'FRACTaL_Y5VF95_0_50ms_0ms_1')
('There is an exception occured at:', 'FRACTaL_Y5VF95_2_50ms_0ms_1')
('There is an exception occured at:', 'FRACTaL_Y5VF95_1_50ms_0ms_1')
('There is an exception occured at:', 'FRACTaL_Y5VF95_4_50ms_0ms_1')
('There is an exception occured at:', 'FRACTaL_N72ZV0_1_50ms_0ms_2')
('There is an exception occured at:', 'FRACTaL_N72ZV0_4_50ms_0ms_2')
('There is an exception occured at:', 'FRACTaL_N72ZV0_3_50ms_0ms_2')
('There is an exception occured at:', 'FRACTaL_N72ZV0_0_50ms_0ms_2')
('There is an exception occured at:', 'FRACTaL_N72ZV0_2_50ms_0ms_2')
('There is an exception occured at:', 'FRACTaL_VQ4FLD_0_50ms_0ms_3')
('There is an exception occured at:', 'FRACTaL_VQ4FLD_3_50ms_0ms_3')
('There is an exception occured at:', 'FRACTaL_VQ4FLD_1_50ms_0ms_3')
('There is an exception occured at:', 'FRACTaL_VQ4FLD_4_50ms_0ms_3')
('There is an exception occured at:', 'FRACTaL_

,1 Subflow,2 Subflows,3 Subflows,5 Subflows
GP [kbps],1780 ± 0,1780 ± 0,1780 ± 0,1780 ± 0
LR [%],14% ± 0%,7% ± 0%,4% ± 0%,2% ± 0%
QMD [ms],540.69 ± 0.0,270.34 ± 0.0,180.23 ± 0.0,108.14 ± 0.0


In [272]:
#######################################################
#                 out-of-phase evaluator - mprtp4
#######################################################

from io import StringIO
import numpy as np
import pandas as pd
from math import log
import math

import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def extract_dataframe(filename, identifier):
    header = False
    content = []
    identifier_length = len(identifier)
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            if identifier not in line:
                continue
            stat_line = line[identifier_length:]
            if "Subflow ID" in stat_line:
                if header is True:
                    continue
                header = True
            content.append(stat_line)
    if len(content) < 1:
        return None
    TESTDATA = StringIO('\n'.join(map(unicode,content)))
    return pd.read_csv(TESTDATA)


class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)


class Subflow:
    def __init__(self, path, stats_df, subflow_id):
        self._path = path
        self._subflow_id = subflow_id
        self._stat = stats_df[stats_df["Subflow ID"] == subflow_id]

        sr = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'subflow_'+ str(subflow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_1_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_1_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp,
            'TE': self._stat["Target Error"].mean()
        }

# The variance around the target is not significantly higher than in case of one subflow
# the ratio stability

base_path = "mprtp4"
debug = False
# debug = True
for path,dirs,files in os.walk(base_path):
    if len(dirs) < 1:
        continue
    final_table = pd.DataFrame()
#     for subflows_num in [1]:
    for subflows_num in [1, 2]:
        suffix = "_" + str(subflows_num)
        metrics = {
            'SR': [],
            'ABU': [],
            'LR': [],
            'LRN': [],            
            'QMD': [],
            'GP': [],
            'TE': [],
        }
        subjects = []
        for directory in dirs:
            if not directory.endswith(suffix):
                continue
            filename = os.path.join(base_path, directory, "mprtpflow_4_" + str(subflows_num) + "-snd.log")
            stats_df = extract_dataframe(filename, "Stat:")
            stats_df["Target Error"] = (stats_df["Target Rate [kbps]"] - stats_df["Sending Rate [kbps]"]) / stats_df["Target Rate [kbps]"]
            subflows = []
            for subflow_id in range(1, subflows_num + 1):
                subflows.append(Subflow(os.path.join(base_path, directory), stats_df, subflow_id))
            
            abs_ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows)
            abs_LR = sum(subflow.get_summary()['LR'] for subflow in subflows)
            abs_GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
            abs_QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows)
            abs_TE = sum(subflow.get_summary()['TE'] for subflow in subflows)
            subjects.append((directory, abs_ABU, abs_LR, abs_GP, abs_QMD, abs_TE))
            
            SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
            ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) / float(subflows_num)
            LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
            LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
            QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
            GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
            TE = sum(subflow.get_summary()['TE'] for subflow in subflows) / float(subflows_num)            
            metrics['SR'].append(SR)
            metrics['ABU'].append(ABU)            
            metrics['LR'].append(LR)
            metrics['LRN'].append(LRN)            
            metrics['QMD'].append(QMD)
            metrics['GP'].append(GP)
            metrics['TE'].append(TE)
        
        # Display parameters
        if debug:
            index = 1
            if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                print("for measuring ', subflows_num, 'subflows, \
                      it does not have the necessary minimum number of measurements")
#             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
            sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
            for item in sorted_list:
                print("Rank:", index)
                print("Directory, ABU, LR, GP, QMD, TE: ", item)
                pdf = PDF(base_path + '/' + item[0] + '/' + 'mprtp4_aggr_FRACTaL_50ms_0ms.pdf')
                display(pdf)
                index += 1
            print('-' * 20)
            
        subflow_col = (str(subflows_num) + ' Subflows') if 1 < subflows_num else '1 Subflow'
        final_table[subflow_col] = list(range(7))
        SRs = np.asarray(metrics['SR'])
        ABUs = np.asarray(metrics['ABU'])
        LRs = np.asarray(metrics['LR'])
        LRNs = np.asarray(metrics['LRN'])
        QMDs = np.asarray(metrics['QMD'])
        GPs = np.asarray(metrics['GP'])
        TEs = np.asarray(metrics['TE'])

        final_table[subflow_col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
        final_table[subflow_col][1] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
        final_table[subflow_col][2] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
        final_table[subflow_col][3] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
        final_table[subflow_col][4] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
        final_table[subflow_col][5] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
        final_table[subflow_col][6] = str(int(TEs.mean() * 100)) + '% ± ' + str(int(TEs.std() * 100) ) + '%'
        final_table[subflow_col][7] = str(len(metrics['SR']))
        
    final_table = final_table.rename({
        0: 'GP [kbps]',
        1: 'ABU [%]', 
        2: 'LR [%]', 
        3: 'LRN',
        4: 'QMD [ms]',
        5: 'SR [kbps]',
        6: 'TE [%]',
        7: 'Samples'}, axis='index')
#     display(final_table)
    display(final_table.loc[['GP [kbps]', 'TE [%]', 'LR [%]', 'QMD [ms]'], :])



/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,1 Subflow,2 Subflows
GP [kbps],1002 ± 88,1844 ± 96
TE [%],6% ± 0%,4% ± 1%
LR [%],0% ± 0%,1% ± 0%
QMD [ms],114.73 ± 2.2,107.43 ± 2.3


In [ ]:
#######################################################
#      shared bottleneck, fairness test - mprtp5
#######################################################
# TODO!
import pandas as pd
import numpy as np

class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

packets_colnames = ["Tracked NTP",
                    "Sequence Number",
                    "Timestamp",
                    "SSRC",
                    "Payload Type",
                    "Payload Size",
                    "Subflow ID",
                    "Subflow Sequence Number",
                    "Header Size",
                    "Protect Begin",
                    "Protect End",
                    "Marker",
                   ]

class Subflow:
    def __init__(self, channel, path, subflow_id):
        self._path = path
        self._subflow_id = subflow_id
        self._df = pd.DataFrame()
        sr = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'subflow_'+ str(subflow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }
        
    
def collect_subflows(channel, path, subflows_num):
    subflows = [Subflow(channel, path, subflow_id) for subflow_id in range(1, subflows_num + 1)]
    return subflows

    
import os
from os import listdir
from os.path import isfile, join

configs = [
{
    'base_path': "mprtp5",
    'debug': False,
#     'debug': True,
    'channel': "2"
}, 
]

for config in configs:
    print(config)
    for path,dirs,files in os.walk(base_path):
        if len(dirs) < 1:
            continue
        final_table = pd.DataFrame()
    #     for subflows_num in [5]:
        for subflows_num in [1, 2, 3, 5]:
    #         print("number of subflows", subflows_num)
            suffix = "_" + str(subflows_num)
            metrics = {
                'SR': [],
                'ABU': [],
                'LR': [],
                'LRN': [],            
                'QMD': [],
                'GP': [],
            }
            subjects = []
            for directory in dirs:
                if not directory.endswith(suffix):
                    continue
                try:
                    subflows = collect_subflows(config['channel'], config['base_path'] + '/' + directory, subflows_num)
                except:
                    print("There is an exception occured at:", directory)

                abs_ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows)
                abs_LR = sum(subflow.get_summary()['LR'] for subflow in subflows)
                abs_GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
                abs_QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows)
                abs_LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)
                subjects.append((directory, abs_ABU, abs_LR, abs_GP, abs_QMD, abs_LRN))

                SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
                ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) / float(subflows_num)
                LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
                LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
                QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
                GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
                metrics['SR'].append(SR)
                metrics['ABU'].append(ABU)            
                metrics['LR'].append(LR)
                metrics['LRN'].append(LRN)            
                metrics['QMD'].append(QMD)
                metrics['GP'].append(GP)

            # Display parameters
            if config['debug']:
                index = 1
                if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                    print("for measuring ', subflows_num, 'subflows, \
                          it does not have the necessary minimum number of measurements")
    #             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
                sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
                for item in sorted_list:
                    print("Rank:", index)
                    print("Directory, ABU, LR, GP, QMD, LRN: ", item)
                    pdf = PDF(base_path + '/' + item[0] + '/' + 'mprtp2_aggr_FRACTaL_50ms_0ms.pdf')
                    display(pdf)
                    index += 1
                print('-' * 20)

            subflow_col = (str(subflows_num) + ' Subflows') if 1 < subflows_num else '1 Subflow'
            final_table[subflow_col] = list(range(7))
            SRs = np.asarray(metrics['SR'])
            ABUs = np.asarray(metrics['ABU'])
            LRs = np.asarray(metrics['LR'])
            LRNs = np.asarray(metrics['LRN'])
            QMDs = np.asarray(metrics['QMD'])
            GPs = np.asarray(metrics['GP'])

            final_table[subflow_col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
            final_table[subflow_col][1] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
            final_table[subflow_col][2] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
            final_table[subflow_col][3] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
            final_table[subflow_col][4] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
            final_table[subflow_col][5] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
            final_table[subflow_col][6] = str(len(metrics['SR']))

        final_table = final_table.rename({
            0: 'GP [kbps]',
            1: 'ABU [%]', 
            2: 'LR [%]', 
            3: 'LRN',
            4: 'QMD [ms]',
            5: 'SR [kbps]',
            6: 'Samples'}, axis='index')
    #     display(final_table)
        display(final_table.loc[['GP [kbps]', 'LR [%]', 'QMD [ms]'], :])

    
# We need the number of lost packets instead!!!!!